## Classification (c)

In [1]:
from neuralnet import *
from random import shuffle
import numpy as np

config_c = {}
config_c['layer_specs'] = [3072, 64, 64, 10]
config_c['activation'] = 'tanh'
config_c['learning_rate'] = 0.005 
config_c['batch_size'] = 128 
config_c['epochs'] = 100  
config_c['early_stop'] = True 
config_c['early_stop_epoch'] = 5  
config_c['L2_penalty'] = 0  
config_c['momentum'] = True  
config_c['momentum_gamma'] = 0.9  
# Create the model

#TODO
#momentum, early stop , expect accuracy around 37%

In [2]:
"""
Train your model here.
Implement batch SGD to train the model.
Implement Early Stopping.
Use config to set parameters for training like learning rate, momentum, etc.
"""
def train(model, x_train, y_train, x_valid, y_valid, config, patience=5):
    epochs = 100#config_c['epochs']
    batch_size = config['batch_size']
    momentum =    config['momentum']
    momentum_gamma = config['momentum_gamma']
    L2_penalty = config['momentum_gamma']


    train_loss_record = []
    train_accuracy_record = []
    holdout_loss_record = []
    holdout_accuracy_record = []

    # How many times the validation loss has gone up in a row.
    cur_loss_up_sequence = 0

    for epoch in range(epochs):
        print(f' epoch: {epoch + 1}')
        batch_loss = []
        batch_accuracy = []
        for x, y in generate_minibatches(x_train, y_train, batch_size):
            # Forward Pass
            y,loss = model.forward(x, y)
            batch_loss.append(loss) 
            # Backward Pass
            model.backward()
            model.updateweight() # update weight for each layer.
            batch_accuracy.append(model.accuracy(x,y))

    
        train_loss = np.mean(batch_loss)
        train_accuracy = np.mean(batch_accuracy)
        
        holdout_loss = model.forward(x_valid, y_valid)[1]
        holdout_accuracy = model.accuracy(x_valid, y_valid)

        train_loss_record.append(train_loss)
        train_accuracy_record.append(train_accuracy)

        holdout_loss_record.append(holdout_loss)
        holdout_accuracy_record.append(holdout_accuracy)

        print(f' epoch: {epoch + 1}, train accuracy: {train_accuracy:.4f}, train_loss_norm:{train_loss:.4f}, '\
            f'valid_acc: {holdout_accuracy:.4f}, valid_loss_norm: {holdout_loss:.4f}')   


        # Save the best weights 
        if holdout_accuracy >= max(holdout_accuracy_record):
            model.save_load_weight(save=True)


    # Recall parameters with minimum validation loss
    model.save_load_weight(save=False)
    test_accuracy = test(model, x_test, y_test)        

    print(test_accuracy)


model_c  = Neuralnetwork(config_c)

# Load the data
x_train, y_train, stats = load_data(path="./data",stats = None, mode="train")
x_test, y_test = load_data(path="./data",stats = stats, mode="test")


# TODO(done): Create splits for validation data here.
x_train, y_train, x_valid, y_valid = split_data(x_train,y_train)
#mini batch stochastic gradient
train(model_c,x_train,y_train,x_valid,y_valid,config_c)



(50000, 32, 32, 3)
inp: (50000, 32, 32, 3)
 epoch: 1
 epoch: 1, train accuracy: 0.9917, train_loss_norm:16.6660, valid_acc: 0.0702, valid_loss_norm: 17.1275
 epoch: 2


ValueError: operands could not be broadcast together with shapes (128,10) (128,3072) 

In [ ]:














plt.figure(1)
plt.plot(recording['epoches'], recording['train_loss'], label='train')
plt.plot(recording['epoches'], recording['valid_loss'], label='validation')
plt.xlabel('Epoches')
plt.ylabel('Loss')
plt.title('Training and validation loss vs number of training epochs')
plt.legend()
plt.show()

plt.figure(2)
plt.plot(recording['epoches'], recording['train_accuracy'], label='train')
plt.plot(recording['epoches'], recording['valid_accuracy'], label='validation')
plt.xlabel('Epoches')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy vs number of training epochs')
plt.legend()
plt.show()